In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# Dataset

Creating a dummy dataset. In this test we are going to make the perceptron infer the function f (x) = 2x + 10. 

In [ ]:
x_train = np.arange(15, dtype=np.float32)
y_train = 2*x_train+10

x_validation = np.arange(18, 34, 2, dtype=np.float32)
y_validation = 2*x_validation+10

x_test = np.arange(50, 55, dtype=np.float32)
y_test = 2*x_test+10

In [ ]:
plt.plot(np.arange(60), 2*np.arange(60)+10, color='red')
plt.scatter(x_train, y_train, label='Training set')
plt.scatter(x_validation, y_validation, color="green", label='Validation set')
plt.scatter(x_test, y_test, color="green", label='Test set')
plt.legend()
plt.show()

**The dataset is divided into three variables**:

**x_train:** training dataset; **y_train:** training targets;

**x_validation:** validation dataset; **y_validation:** targets of validation;

**x_test:** testing dataset; **y_test:** targets from test; 


### Datos ruidosos

In [ ]:
mu, sigma = 0, 1.2 # mean and standard deviation
gauss_noise = np.array(np.random.normal(mu, sigma, x_train.size), dtype=np.float32).reshape(x_train.shape)
y_train_noisy = y_train + gauss_noise

plt.scatter(x_train, y_train_noisy)
plt.plot(x_train, y_train, color='red')
plt.show()

**y_train_noisy** contains the training *targets* by applying a Gaussian noise to it. 

### Preparación del dataset


In [ ]:
dataset = {'train': [x_train, y_train], 
           'validation': [x_validation, y_validation], 
           'test': [x_test, y_test]}

dataset_noisy = {'train': [x_train, y_train_noisy], 
                 'validation': [x_validation, y_validation], 
                 'test': [x_test, y_test]}

# Structure of the Perceptron and training 

In [ ]:
import torch
import torch.nn as nn

class Perceptron(nn.Module):
    def __init__(self, input_size, output_size=1):
        super().__init__()
        self.linear = torch.nn.Linear(input_size, output_size, bias=True)
        self.activation = torch.nn.Identity()
        
    def forward(self, x):
        result = self.linear(x)
        return self.activation(result)

In [ ]:
import copy 

def npToTensor(x):
    return torch.tensor(x.reshape(-1, 1))

def train(dataset, model, optimizer, criterion, num_epochs=25):
    best_model_weigths = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    
    device = torch.device("cpu")
    if torch.cuda.is_available:
        device=torch.device("cuda")  
    
    for epoch in range(num_epochs):
        for phase in ['train', 'validation']:
            inputs = npToTensor(dataset[phase][0]).to(device)
            targets = npToTensor(dataset[phase][1]).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, targets)
            if (phase == 'train'):
                # get gradients w.r.t to parameters
                loss.backward()

                # update parameters
                optimizer.step()
            else:
                # LOG
                if (epoch % 50 == 0):
                    print('epoch {}, loss {}'.format(epoch, loss.item()))
                    
                if (loss.item() < best_loss):
                    #print('\tepoch {}, validation loss {}'.format(epoch, loss.item()))
                    best_loss = loss.item()
                    best_model_weigths = copy.deepcopy(model.state_dict())
        
    print('epoch {}, loss {}'.format(epoch, loss.item()))
    print('Best loss: {}'.format(best_loss))
            
    model.load_state_dict(best_model_weigths)

We create our perceptron... 

In [ ]:
input_dim = 1        # takes variable 'x' 
output_dim = 1       # takes variable 'y'

device=torch.device("cpu")
if torch.cuda.is_available:
    device=torch.device("cuda")

model = Perceptron(input_dim, output_dim).to(device)

In [ ]:
learningRate = 0.001
epochs = 10000
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

train(dataset, model, optimizer, criterion, num_epochs=epochs)

In [ ]:
inputs = npToTensor(dataset['test'][0]).to(device)
result = model(inputs).cpu().detach().numpy()

plt.scatter(dataset['test'][0], dataset['test'][1])
plt.plot(dataset['test'][0], result, color='red')
plt.show()

# Prueba con el dataset ruidoso

In [ ]:
model_noisy = Perceptron(input_dim, output_dim).to(device)

In [ ]:
learningRate = 0.001
epochs = 1000
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model_noisy.parameters(), lr=learningRate)

train(dataset_noisy, model_noisy, optimizer, criterion, num_epochs=epochs)

In [ ]:
inputs = npToTensor(dataset['test'][0]).to(device)
result = model_noisy(inputs).cpu().detach().numpy()

plt.scatter(dataset['test'][0], dataset['test'][1])
plt.plot(dataset['test'][0], result, color='red')
plt.show()

# Modulo Dataset en PyTorch

In [ ]:
from torch.utils.data import Dataset
from torch import tensor

class LinearDataset(Dataset):
    '''
        Dataset f(x)=2x+10
    '''
    def __init__(self, start, input_size=12, step=1, noise=False):
        self.noise = noise
        self.mu, self.sigma = 0, 0.5 # mean and standard deviation
        self.x_values = np.arange(start, start + input_size*step, step, dtype=np.float32).reshape(-1,1)
        self.gauss_noise = np.array(np.random.normal(self.mu, self.sigma, self.x_values.size), dtype=np.float32).reshape(self.x_values.shape)
           
    def __len__(self):
        return self.x_values.size
    
    def __getitem__(self, idx):
        x = self.x_values[idx]
        y = (2*x + 10)
        
        if (self.noise):
            apply = np.random.uniform() > 0.75
            y = y + apply*self.gauss_noise[idx]
            
        return [tensor(x), tensor(y)]

In [ ]:
from torch.utils.data import DataLoader

train_set = LinearDataset(0, 50, 1, True)
validation_set = LinearDataset(30, 8, 2)

batch_size = 12

dataloaders = {
    'train': DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0),
    'validation': DataLoader(validation_set, batch_size=batch_size, shuffle=True, num_workers=0),
}

for inputs, labels in dataloaders['train']:
    print(inputs.size())   

In [ ]:
def train2(dataloaders, model, optimizer, criterion, num_epochs=25):
    best_model_weigths = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    
    device = torch.device("cpu")
    if torch.cuda.is_available:
        device=torch.device("cuda")  
    
    for epoch in range(num_epochs):
        for phase in ['train', 'validation']:
            for inputs, targets in dataloaders[phase]:
                inputs = inputs.to(device)
                targets = targets.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)

                loss = criterion(outputs, targets)
                if (phase == 'train'):
                    # get gradients w.r.t to parameters
                    loss.backward()

                    # update parameters
                    optimizer.step()
                else:
                    # LOG
                    if (epoch % 50 == 0):
                        print('epoch {}, loss {}'.format(epoch, loss.item()))

                    if (loss.item() < best_loss):
                        #print('\tepoch {}, validation loss {}'.format(epoch, loss.item()))
                        best_loss = loss.item()
                        best_model_weigths = copy.deepcopy(model.state_dict())
        
    print('epoch {}, loss {}'.format(epoch, loss.item()))
    print('Best loss: {}'.format(best_loss))
            
    model.load_state_dict(best_model_weigths)

In [ ]:
another_model = Perceptron(input_dim, output_dim).to(device)

In [ ]:
learningRate = 0.001
epochs = 1000
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(another_model.parameters(), lr=learningRate)

train2(dataloaders, another_model, optimizer, criterion, num_epochs=epochs)

In [ ]:
inputs = npToTensor(dataset['test'][0]).to(device)
result = another_model(inputs).cpu().detach().numpy()

plt.scatter(dataset['test'][0], dataset['test'][1])
plt.plot(dataset['test'][0], result, color='red')
plt.show()